In [1]:
import customtkinter
from tkintermapview import TkinterMapView
from PIL import Image
import os
import thingspeak
import numpy as np
import warnings
import operator
import re
import requests
from datetime import datetime
from pytz import timezone
import string

Device_Lookup = {
    'Device 1': 2496188,
    'Device 2': 2496183,
    'Device 3': 2496176
    #'Device 4': 2496176,
    #'Device 5': 2496176,
    #'Device 6': 2496176,
    #'Device 7': 2496176,
    #'Device 8': 2496176
    }

Message_Decoder = {
    0: "",
    1: "Task complete",
    2: "Starting Task",
    3: "Need a hand",
    4: "Repair Needed",
    5: "Pest Sighting",
    6: "Equipment Malfunction",
    7: "Deliveries Arrived"
}

In [2]:
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

def extract_vals(ChannelID, field):
    
    ch = thingspeak.Channel(ChannelID)
    
    field_out = ch.get_field_last(field)
    if field == "field1":
        field_out = field_out.split("field1",1)[1]
    if field == "field2":
        field_out = field_out.split("field2",1)[1]
    temp = re.findall(r'\d+', field_out)
    if int(temp[0]) == 0:
        return 0
    temp[1]=temp[1][0:6]
    while(int(temp[1])<100000):
        temp[1] = int(temp[1]) *10
    field_out = list(map(int, temp))
    field_out = field_out[0] + field_out[1]*10**-6
    if (field == 'field1'):
        field_out = -field_out
        
    return field_out

def extract_times(ChannelID, field):
    
    ch = thingspeak.Channel(ChannelID)
    
    field_out = ch.get_last_data_age(field)
    temp = re.findall(r'\d+', field_out)
    field_out = list(map(int, temp))
    t = (field_out[0])
        
    return t

def format_time(t):

    hours = np.floor(t/3600)
    minutes = t%3600
    minutes = np.floor(minutes/60)
    if t < 60:
        seconds = t
    else: 
        seconds = t%(hours*3600 + minutes*60)
        
    return hours, minutes, seconds

def getThingSpeakDataOverTime(channelID, field):

    url = "https://api.thingspeak.com/channels/{}/feeds.json".format(channelID)
    get_data = requests.get(url).json()

    field_1 = get_data['feeds']

    t=[]
    for x in field_1:
        t.append(x[field])
        
    return t

def getGPSarray(string):
    data = [[] for _ in range(2*len(string))]
    i=0
    for x in string:
        data[i] = getThingSpeakDataOverTime(Device_Lookup[x], 'field1')
        i=i+1
        data[i] = getThingSpeakDataOverTime(Device_Lookup[x], 'field2')
        i=i+1

    for i in range(2*len(string)):
        x=0
        for x in range(int(np.size(data)/(2*len(string)))):
            if data[i][x] == None:
                data[i][x] = 0
    
    vals = []

    # List of devices to check
    devices = ['Device 1', 'Device 2', 'Device 3', 'Device 4', 'Device 5', 'Device 6','Device 7','Device 8']

    for i, device in enumerate(devices, start=1):
        if device in string:
            vals.append(i)
    
    return data, vals

def gettoday():
    
    tz = timezone('GMT')
    today = datetime.now(tz)

    if today.month < 10 and today.day >= 10:
        date_string = "{}-0{}-{}".format(today.year, today.month, today.day)
    if today.month < 10 and today.day < 10:
        date_string = "{}-0{}-0{}".format(today.year, today.month, today.day)
    if today.month >= 10 and today.day >= 10:
        date_string = "{}-{}-{}".format(today.year, today.month, today.day)
    if today.month >= 10 and today.day < 10:
        date_string = "{}-{}-{}".format(today.year, today.month, today.day)
        
    return date_string
            
            
def getGPSarray_on_day(string, string_in): #string_in is either "current_day" or a specified dat in format year-month-day
    tz = timezone('GMT')
    today = datetime.now(tz)

    if string_in == "current_day":
        if today.month < 10 and today.day >= 10:
            date_string = "{}-0{}-{}".format(today.year, today.month, today.day)
        if today.month < 10 and today.day < 10:
            date_string = "{}-0{}-0{}".format(today.year, today.month, today.day)
        if today.month >= 10 and today.day >= 10:
            date_string = "{}-{}-{}".format(today.year, today.month, today.day)
        if today.month >= 10 and today.day < 10:
            date_string = "{}-{}-{}".format(today.year, today.month, today.day)
    else:
        date_string = string_in 

    times = [[] for _ in range(len(string))]
    i=0
    for x in string:
        times[i] = getThingSpeakDataOverTime(Device_Lookup[x], 'created_at')
        i=i+1

    total_data_points_array = np.zeros(len(times))
    starting_index_array = np.zeros(len(times))
    ending_index_array = np.zeros(len(times))

    for i in range(len(times)):
        for k in range(len(times[i])):
            if date_string in times[i][k]:
                starting_index_array[i] = k
                break

        for j in range(len(times[i])):
            if date_string in times[i][j]:
                total_data_points_array[i] = total_data_points_array[i] + 1

    for i in range(len(times)):
        ending_index_array[i] = starting_index_array[i] + total_data_points_array[i]

    data = [[] for _ in range(2*len(string))]
    i=0
    for x in string:
        data[i] = getThingSpeakDataOverTime(Device_Lookup[x], 'field1')
        i=i+1
        data[i] = getThingSpeakDataOverTime(Device_Lookup[x], 'field2')
        i=i+1

    for i in range(2*len(string)):
        x=0
        for x in range(int(np.size(data)/(2*len(string)))):
            if data[i][x] == None:
                    data[i][x] = 0

    i=0
    for k in range(len(string)):
        data[i] = data[i][int(starting_index_array[k]):int(ending_index_array[k])]
        i = i+1
        data[i] = data[i][int(starting_index_array[k]):int(ending_index_array[k])]
        i = i+1
        
    vals = []

    # List of devices to check
    devices = ['Device 1', 'Device 2', 'Device 3', 'Device 4', 'Device 5', 'Device 6','Device 7','Device 8']

    num_devices = 0
    for i, device in enumerate(devices, start=1):
        if device in string:
            num_devices = num_devices + 1
            if data[(num_devices-1)*2] != []:
                vals.append(i)

    return data, vals

def get_todays_messages(ChannelID):
    tz = timezone('GMT')
    today = datetime.now(tz)

    if today.month < 10 and today.day >= 10:
        date_string = "{}-0{}-{}".format(today.year, today.month, today.day)
    if today.month < 10 and today.day < 10:
        date_string = "{}-0{}-0{}".format(today.year, today.month, today.day)
    if today.month >= 10 and today.day >= 10:
        date_string = "{}-{}-{}".format(today.year, today.month, today.day)
    if today.month >= 10 and today.day < 10:
        date_string = "{}-{}-{}".format(today.year, today.month, today.day)

    #date_string = "2024-04-14" #for testing
    
    times = getThingSpeakDataOverTime(ChannelID, 'created_at')
    
    total_data_points = 0
    starting_index = 0
    ending_index = 0

    for k in range(len(times)):
        if date_string in times[k]:
            starting_index = k
            break

    for j in range(len(times)):
            if date_string in times[j]:
                total_data_points = total_data_points + 1

    ending_index = starting_index + total_data_points

    data = getThingSpeakDataOverTime(ChannelID, 'field5')

    x=0
    for i in range(len(data)):
        if data[i] == None:
            data[i] = 0
        else:
            data[i] = int(data[i])

    data = data[int(starting_index):int(ending_index)]

    unique = []

    for i in data:
        if i not in unique and i != 0:
            unique.append(i)
    
    unique_message_code = 0
    for i, vals in enumerate(unique, start = 1):
        unique_message_code = unique_message_code + 10**(i-1)*vals
            
    return unique_message_code

def getGPSarraysFormatted(data):
    arrays = []

    for i in range(0, len(data), 2):
        
        temp_array = []
        for j in range(np.size(data[i])):
            temp_array.append(data[i][j])
            
            if i + 1 < len(data):
                temp_array.append(data[i+1][j])

        temp_np_array = np.array(temp_array).reshape(-1, 2)
        arrays.append(temp_np_array)

    return arrays

def get_latest_voltage(ChannelID): 

    ch = thingspeak.Channel(ChannelID)
    voltage = ch.get_field_last('field4')

    voltage = voltage.split("field4",1)[1]
    voltage = re.findall("\d+\.\d+", voltage)
    if voltage == []:
        voltage=0
    else:
        voltage=float(voltage[0])

    return voltage

def get_emergency_status(ChannelID):

    ch = thingspeak.Channel(ChannelID)
    status = ch.get_field_last('field3')
    status = status.split("field3",1)[1]
    temp = re.findall(r'\d+', status)
    status = list(map(int, temp))
    status = status[0]
    
    return status

def get_boundary_status(ChannelID):

    ch = thingspeak.Channel(ChannelID)
    status = ch.get_field_last('field6')
    if "field6" in status:
        status = status.split("field6",1)[1]
        temp = re.findall(r'\d+', status)
        status = list(map(int, temp))
        status = status[0]
    else:
        status = 0
    
    return status

def get_message_code(ChannelID):

    ch = thingspeak.Channel(ChannelID)
    status = ch.get_field_last('field5')
    if "field5" in status:
        status = status.split("field5",1)[1]
        temp = re.findall(r'\d+', status)
        status = list(map(int, temp))
        status = status[0]
    else:
        status = 0
    
    return status

In [3]:
customtkinter.set_default_color_theme("blue")

class MyScrollableCheckboxFrame(customtkinter.CTkScrollableFrame):
    def __init__(self, master, title, values):
        super().__init__(master, label_text=title)
        self.grid_columnconfigure(0, weight=1)
        self.values = values
        self.checkboxes = []

        for i, value in enumerate(self.values):
            checkbox = customtkinter.CTkCheckBox(self, text=value)
            checkbox.select()
            checkbox.grid(row=i, column=0, padx=10, pady=(10, 0), sticky="w")
            self.checkboxes.append(checkbox)

    def get(self):
        checked_checkboxes = []
        for checkbox in self.checkboxes:
            if checkbox.get() == 1:
                checked_checkboxes.append(checkbox.cget("text"))
        return checked_checkboxes
    
class MyProgressBarFrame(customtkinter.CTkScrollableFrame):
    def __init__(self, master, title, values, progress, max_volt, min_volt):
        super().__init__(master, label_text=title)
        self.grid_columnconfigure(0, weight=1)
        self.values = values
        self.progress = progress
        self.max_volt = max_volt
        self.min_volt = min_volt
        self.width = 180
        self.height = 10
        self.progressbars = []
        self.labels1 = []
        self.labels2 = []
        self.m = 1/(max_volt-min_volt)
        self.b = self.m*min_volt
        
        for i, value in enumerate(self.values):
            
            percent_charge = max(np.round((self.m*progress[i]-self.b)*100, 1), 0)
            bat_text = "{}% Charge".format(percent_charge)
            label1 = customtkinter.CTkLabel(self, text=bat_text, anchor="e")
            label1.grid(row=i, column=0, padx=10, pady=(0, 0), sticky="e")
            self.labels1.append(label1)
            
            label2 = customtkinter.CTkLabel(self, text=value, anchor="w")
            label2.grid(row=i, column=0, padx=10, pady=(0, 0), sticky="w")
            self.labels2.append(label2)
            
            if percent_charge < 30:
                progress_color = 'red2'
            else:
                progress_color = 'green2'
            progressbar = customtkinter.CTkProgressBar(self, width=self.width, height = self.height, 
                                                       border_color='honeydew2', progress_color=progress_color)
            progressbar.grid(row=i, column=0, padx=10, pady=(25, 0), sticky="w")
            progressbar.set(percent_charge/100)
            self.progressbars.append(progressbar)
            
class MyMessageBarFrame(customtkinter.CTkScrollableFrame):
    def __init__(self, master, title, values, message_code):
        super().__init__(master, label_text=title)
        self.grid_columnconfigure(0, weight=1)
        self.values = values
        self.message_code = message_code
        self.labels = []
        self.textboxes = []
        
        for i, value in enumerate(self.values):
            label = customtkinter.CTkLabel(self, text="{}:".format(value), anchor="w")
            label.grid(row=2*i, column=0, padx=10, pady=(0, 0), sticky="w")
            self.labels.append(label)
            
            textbox = customtkinter.CTkTextbox(self, height=1,width=180, fg_color = 'bisque4',# 'gray17'
                                                          border_color = 'gray17', border_width = 0)
            textbox.grid(row=2*i+1, column=0, padx=10, pady=(0, 5), sticky="w")
            for k in Message_Decoder:
                if '{}'.format(k) in str(message_code[i]):
                    string = "{}\n".format(Message_Decoder[k])
                    textbox.insert(index = "1.0",text=string)
            self.textboxes.append(textbox)
                
class MyEmergencyStatusFrame(customtkinter.CTkScrollableFrame):
    def __init__(self, master, title, values, statuses, image1, image2):
        super().__init__(master, label_text=title)
        #self.grid_columnconfigure(0, weight=1)
        self.values = values
        self.statuses = statuses
        self.image1 = image1
        self.image2 = image2
        self.labels = []
        self.images = []
        
        for i, value in enumerate(self.values):
            label = customtkinter.CTkLabel(self, text=value, anchor="w")
            label.grid(row=i, column=0, padx=40, pady=(10, 0), sticky="w")
            self.labels.append(label)
            
            if self.statuses[i] == 1:
                image = customtkinter.CTkLabel(self, image=self.image1, text="")
            else:
                image = customtkinter.CTkLabel(self, image=self.image2, text="")
            image.grid(row=i, column=1, padx=0, pady=(10, 0), sticky="w")
            self.images.append(image)
            
class MyElapsedTimeFrame(customtkinter.CTkScrollableFrame):
    def __init__(self, master, title, values, times):
        super().__init__(master, label_text=title)
        #self.grid_columnconfigure(0, weight=1)
        self.values = values
        self.times = times
        self.labels1 = []
        self.labels2 = []
        
        for i, value in enumerate(self.values):
            
            hours, minutes, seconds = format_time(times[i])
            
            label1 = customtkinter.CTkLabel(self, text="{}:".format(value), anchor="w")
            label1.grid(row=i, column=0, padx=20, pady=(10, 0), sticky="w")
            self.labels1.append(label1)
            
            label2 = customtkinter.CTkLabel(self, text="{}h {}m {}s".format(hours, minutes, seconds), anchor="w")
            label2.grid(row=i, column=1, padx=0, pady=(10, 0), sticky="w")
            self.labels2.append(label2)
        

In [4]:
class App(customtkinter.CTk):

    APP_NAME = "LoRaWAN Asset Tracker"
    WIDTH = 1200
    HEIGHT = 700
    toggle_flag = 1
    #values=["Device 1"]
    values=["Device 1", "Device 2", "Device 3"]
    #values=['Device 1', 'Device 2', 'Device 3', 'Device 4', 'Device 5', 'Device 6','Device 7','Device 8']
    
    def set_polygons(self, polygons_with_styles):
        
        for polygon, style in polygons_with_styles:
            self.map_widget.set_polygon(
                            polygon,
                            fill_color=style.get('fill_color'),
                            outline_color=style.get('outline_color'),
                            border_width=style.get('border_width', 3),
                            name = style.get('name'),
                            command=self.polygon_click)

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

        self.title(App.APP_NAME)
        self.geometry(str(App.WIDTH) + "x" + str(App.HEIGHT))
        self.minsize(App.WIDTH, App.HEIGHT)
        
        warning_image_path = os.path.join(os.getcwd(), "warning.jpg")
        warning_image = customtkinter.CTkImage(Image.open(warning_image_path),
                                  size=(17, 15))
        
        checkmark_image_path = os.path.join(os.getcwd(), "checkmark.jpg")
        checkmark_image = customtkinter.CTkImage(Image.open(checkmark_image_path),
                                  size=(20, 15))

        self.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.createcommand('tk::mac::Quit', self.on_closing)

        self.marker_list = []

        # ============ create two CTkFrames ============

        self.grid_columnconfigure(0, weight=0)
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)

        self.frame_left = customtkinter.CTkFrame(master=self, width=200, corner_radius=0, fg_color=None)
        self.frame_left.grid(row=0, column=0, padx=0, pady=0, sticky="nsew")

        self.frame_right = customtkinter.CTkFrame(master=self, corner_radius=0)
        self.frame_right.grid(row=0, column=1, rowspan=1, pady=0, padx=0, sticky="nsew")

        # ============ frame_left ============

        self.frame_left.grid_rowconfigure(3, weight=1)
        title_font=customtkinter.CTkFont(family='Arial', size=16, underline=True, weight='bold')
        text_font=customtkinter.CTkFont(family='Arial', size=14, underline=False, weight='bold')
        
        # create tabview
        self.tabview = customtkinter.CTkTabview(self.frame_left, width=200, anchor="w")
        self.tabview.grid(row=0, column=0, padx=(20, 20), pady=(20, 0))
        self.tabview.add("GeoMap")
        self.tabview.add("Properties")
        self.tabview.add("Statuses")
        self.tabview.add("Messages")
        
        values=self.values
        self.checkbox_frame_1 =  MyScrollableCheckboxFrame(self.tabview.tab("GeoMap"), title="LoRaWAN Devices", values=values)
        self.checkbox_frame_1.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        
        self.button_1 = customtkinter.CTkButton(master=self.tabview.tab("GeoMap"),
                                                text="Generate Latest",
                                                command=self.map_latest_button_callback)
        self.button_1.grid(pady=(20, 0), padx=(20, 20), row=2, column=0)
        
        self.button_2 = customtkinter.CTkButton(master=self.tabview.tab("GeoMap"),
                                                text="Generate Path For",
                                                command=self.map_path_button_callback)
        self.button_2.grid(pady=(20, 0), padx=(20, 20), row=3, column=0)
        
        self.path_textbox = customtkinter.CTkTextbox(master=self.tabview.tab("GeoMap"), height = 10, width = 80,
                                                     fg_color = 'gray16')
        self.path_textbox.grid(pady=(0, 0), padx=(20, 20), row=4, column=0)
        date = gettoday()
        self.path_textbox.insert(index = "1.0",text = date)
        
        self.field_textbox = customtkinter.CTkTextbox(self.tabview.tab("GeoMap"), height=10, fg_color = 'gray17', 
                                                          border_color = 'gray17', border_width = 2, font = text_font)
        self.field_textbox.grid(row=5, column=0, padx=(20, 20), pady=(15, 0))
        
        self.label_emergencies = customtkinter.CTkLabel(self.frame_left, text="Require Assistance:", anchor="w", font = title_font)
        self.label_emergencies.grid(row=1, column=0, padx=(20, 20), pady=(20, 0))
        
        self.emergency_textbox = customtkinter.CTkTextbox(self.frame_left, height=70, fg_color = 'DeepSkyBlue3', 
                                                          border_color = 'turquoise3', border_width = 2, font = text_font)
        self.emergency_textbox.grid(row=2, column=0, padx=(20, 20), pady=(0, 0))

        self.map_label = customtkinter.CTkLabel(self.frame_left, text="Display Options:", anchor="w")
        self.map_label.grid(row=4, column=0, padx=(20, 20), pady=(20, 0))
        self.map_option_menu = customtkinter.CTkOptionMenu(self.frame_left, values=["OpenStreetMap", "Google normal", "Google satellite"],
                                                                       command=self.change_map)
        self.map_option_menu.grid(row=8, column=0, padx=(20, 20), pady=(10, 20))
        
        voltages = []
        for i in Device_Lookup:
            voltage = get_latest_voltage(Device_Lookup[i])
            voltages.append(voltage)
        
        self.batterybars = MyProgressBarFrame(self.tabview.tab("Properties"), title = "Device Charge", values=values,
                                             progress = voltages, max_volt = 4.85, min_volt = 4.55)
        self.batterybars.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        
        times = []
        for i in Device_Lookup:
            time = extract_times(Device_Lookup[i], 'field1')
            times.append(time)
            
        self.elapsedframe = MyElapsedTimeFrame(self.tabview.tab("Properties"), title = "Time Since Receive", values=values,
                                             times = times)
        self.elapsedframe.grid(pady=(20, 0), padx=(20, 20), row=2, column=0)
        
        self.button_3 = customtkinter.CTkButton(self.tabview.tab("Properties"),
                                                text="Get Latest",
                                                command=self.voltage_callback)
        self.button_3.grid(pady=(20, 0), padx=(20, 20), row=3, column=0)
        
        statuses = []
        boundary_statuses = []
        for i in Device_Lookup:
            status = get_emergency_status(Device_Lookup[i])
            statuses.append(status)
            
            boundary_status = get_boundary_status(Device_Lookup[i])
            boundary_statuses.append(boundary_status)
        
        #statuses = [0,1,0,0,0,0,0,0]
        #boundary_statuses = [0,1,1,0,0,1,0,0]
        
        total_statuses = np.zeros(len(Device_Lookup))
        for i in range(len(Device_Lookup)):
            total_statuses[i] = statuses[i] + boundary_statuses[i]
            
        total_em = 0
        device_hone = 0
        for i, val in enumerate(total_statuses):
            if val > 0:
                total_em = total_em+1
                string = "               Device {} \n".format(i+1)
                self.emergency_textbox.insert(index = "1.0",text=string)
                device_hone = i+1
                
        if total_em == 0:
            self.emergency_textbox.insert(index = "1.0",text="               No Device \n      Requires Assistance")
        
        self.emergencyframe = MyEmergencyStatusFrame(self.tabview.tab("Statuses"), title = "Emergency Statuses", 
                                                     values = values, statuses = statuses, 
                                                     image1=warning_image, image2=checkmark_image)
        self.emergencyframe.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        
        self.outofboundframe = MyEmergencyStatusFrame(self.tabview.tab("Statuses"), title = "Boundary Status", 
                                                     values = values, statuses = boundary_statuses, 
                                                     image1=warning_image, image2=checkmark_image)
        self.outofboundframe.grid(pady=(20, 0), padx=(20, 20), row=2, column=0)
        
        self.emergencytext1 = customtkinter.CTkLabel(self.tabview.tab("Statuses"), text="No Action Needed: ", anchor="w")
        self.emergencytext1.grid(row=3, column=0, padx=0, pady=(10, 0), sticky="w")
        
        self.emergencytext1 = customtkinter.CTkLabel(self.tabview.tab("Statuses"), image=checkmark_image,text="")
        self.emergencytext1.grid(row=3, column=0, padx=110, pady=(10, 0), sticky="w")
        
        self.emergencytext1 = customtkinter.CTkLabel(self.tabview.tab("Statuses"), text="Assistance Required: ", anchor="w")
        self.emergencytext1.grid(row=4, column=0, padx=0, pady=(10, 0), sticky="w")
        
        self.emergencytext1 = customtkinter.CTkLabel(self.tabview.tab("Statuses"), image=warning_image,text="")
        self.emergencytext1.grid(row=4, column=0, padx=125, pady=(10, 0), sticky="w")
        
        self.button_4 = customtkinter.CTkButton(self.tabview.tab("Statuses"),
                                                text="Get Latest",
                                                command=self.status_callback)
        self.button_4.grid(pady=(20, 0), padx=(20, 20), row=5, column=0)
        
        messages = []
        for i in Device_Lookup:
            code = get_todays_messages(Device_Lookup[i])
            messages.append(code)
            
        #messages = [0,1,0,2,0,0,2,0]
            
        self.MessageFrame = MyMessageBarFrame(self.tabview.tab("Messages"), title = "Received Messages", 
                                             values = values, message_code = messages)
        self.MessageFrame.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        
        self.button_aaaaaaaa = customtkinter.CTkButton(self.tabview.tab("Messages"),
                                                text="Get Latest",
                                                command=self.message_callback)
        self.button_aaaaaaaa.grid(pady=(20, 0), padx=(20, 20), row=2, column=0)

        # ============ frame_right ============

        self.frame_right.grid_rowconfigure(1, weight=1)
        self.frame_right.grid_rowconfigure(0, weight=0)
        self.frame_right.grid_columnconfigure(0, weight=1)
        self.frame_right.grid_columnconfigure(1, weight=0)
        self.frame_right.grid_columnconfigure(2, weight=1)

        self.map_widget = TkinterMapView(self.frame_right, corner_radius=0)
        self.map_widget.grid(row=1, rowspan=1, column=0, columnspan=4, sticky="nswe", padx=(0, 0), pady=(0, 0))

        self.entry = customtkinter.CTkEntry(master=self.frame_right,
                                            placeholder_text="type address")
        self.entry.grid(row=0, column=0, sticky="we", padx=(12, 0), pady=12)
        self.entry.bind("<Return>", self.search_event)

        self.button_5 = customtkinter.CTkButton(master=self.frame_right,
                                                text="Search",
                                                width=90,
                                                command=self.search_event)
        self.button_5.grid(row=0, column=1, sticky="w", padx=(12, 0), pady=12)
        
        self.button_toggle = customtkinter.CTkButton(master=self.frame_right,
                                                text="Toggle Polygons",
                                                command=self.toggle_button_callback)
        self.button_toggle.grid(row=0, column=2, sticky="e", padx=(12, 0), pady=12)
        
        self.deviceoptionmenu = customtkinter.CTkOptionMenu(master=self.frame_right, 
                                                            values=values,
                                                             command=self.deviceoptionmenu_callback)
        self.deviceoptionmenu.grid(row=0, column=3, sticky="e", padx=(12, 12), pady=12)

        # Set default values
        self.map_widget.set_tile_server("https://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}&s=Ga", max_zoom=22)
        self.map_widget.set_address("Quincy, FL")
        self.map_option_menu.set("Google satellite")
        
        if device_hone != 0:
            channelID = Device_Lookup["Device {}".format(device_hone)]
            latitude = extract_vals(channelID, 'field2')
            longitude = extract_vals(channelID, 'field1')
            if latitude != 0 and longitude != 0:
                self.map_widget.set_position(latitude, longitude)
                self.map_widget.set_zoom(18)
                self.map_widget.set_marker(latitude, longitude, text = "Device {} Requires Assistance!".format(device_hone),
                                                       marker_color_circle = 'red',
                                                       marker_color_outside= 'red2',text_color = 'white')
            
        else:
            self.map_widget.set_position(30.545218, -84.593958)
            self.map_widget.set_zoom(16)
            
        self.map_widget.add_right_click_menu_command(label="Print Coordinates",
                                        command=self.add_marker_event,
                                        pass_coords=True)
        
        self.toggle_polygons(0)

    def search_event(self, event=None):
        self.map_widget.set_address(self.entry.get())

    def change_appearance_mode(self, new_appearance_mode: str):
        customtkinter.set_appearance_mode(new_appearance_mode)
        
    def add_marker_event(self, coordinates_tuple):
        print(coordinates_tuple)
        
    def deviceoptionmenu_callback(self, choice):
        channelID = Device_Lookup[choice]
        latitude = extract_vals(channelID, 'field2')
        longitude = extract_vals(channelID, 'field1')
        if longitude != 0 and latitude != 0:
            self.map_widget.set_position(latitude, longitude)
            self.map_widget.set_zoom(18)
        
    def polygon_click(self, polygon):
        self.field_textbox.delete("0.0", "end")
        self.field_textbox.insert(index = "1.0",text="                 {}".format(polygon.name))
        
    def toggle_polygons(self, index):
        field1 = [(30.545141942063744, -84.59275483150049),
                    (30.54676399593058, -84.59221381078791),
                    (30.548173740954006, -84.59198045860362),
                    (30.548725811243415, -84.59191876779627),
                    (30.54876276981176, -84.58535324389369),
                    (30.5414743327946, -84.58538274819286),
                    (30.541467402543923, -84.5889138670436),
                    (30.544930156190215, -84.58895678238784),
                    (30.545082616154286, -84.58913649039184),
                    (30.545099713176366, -84.592749161245)]
        
        field2 = [(30.548488367331014, -84.6003616389747),
                    (30.548513776410406, -84.60055570862514),
                    (30.54874476773621, -84.60079174301845),
                    (30.54870780916102, -84.6010170485757),
                    (30.548726881003738, -84.60132818482143),
                    (30.548874715191047, -84.6015213038705),
                    (30.55107372214652, -84.60147838852626),
                    (30.55101828560845, -84.60114579460841),
                    (30.551092200985497, -84.60087757370692),
                    (30.551156876894247, -84.60081320069057),
                    (30.551286228582473, -84.6009097602151),
                    (30.55131394677895, -84.601070692756),
                    (30.55153149536629, -84.60139381031992),
                    (30.5521412923197, -84.60154401402475),
                    (30.55227064269612, -84.59851848225595),
                    (30.552501625082414, -84.59846483807566),
                    (30.552473907225085, -84.59493505101206),
                    (30.549831435148093, -84.59478484730722),
                    (30.549803716528295, -84.59596501927378),
                    (30.549314019606815, -84.5959864769459),
                    (30.54939717586194, -84.59947334866526),
                    (30.549212384087145, -84.59979521374704),
                    (30.54889823726271, -84.59998833279612),
                    (30.54863952734986, -84.60000979046823),
                    (30.54851941179882, -84.60032092671396)]
        
        field3 = [(30.54161511069401, -84.59447828124564),
                    (30.541615574288333, -84.59720340560477),
                    (30.542317836106484, -84.59725168536704),
                    (30.542465680052633, -84.59831920455497),
                    (30.543177175895188, -84.59806171248954),
                    (30.543490460495693, -84.60294341038863),
                    (30.54497811197472, -84.60305865167693),
                    (30.54504279195677, -84.60149224161223),
                    (30.54550625838856, -84.60176216684381),
                    (30.54576497665091, -84.60215913377802),
                    (30.546781363150245, -84.60135447107355),
                    (30.54740042971299, -84.60131396159925),
                    (30.547760780073656, -84.60146416530408),
                    (30.54791785545332, -84.60116375789441),
                    (30.547723821123864, -84.60074533328809),
                    (30.54737271039908, -84.60054148540296),
                    (30.546522647596273, -84.59956516132154),
                    (30.546624285931834, -84.59942568645276),
                    (30.547261833064276, -84.5998655687312),
                    (30.547455868317073, -84.59976900920667),
                    (30.547880896563317, -84.60014451846875),
                    (30.548148848197027, -84.59969390735425),
                    (30.548010252616752, -84.59925402507581),
                    (30.547631423687818, -84.5990524242384),
                    (30.54746510808134, -84.59875460682629),
                    (30.547326511524965, -84.59763497840609),
                    (30.54733575130155, -84.59591499061717),
                    (30.548601592380383, -84.59586134643688),
                    (30.548650613833644, -84.59223172164067),
                    (30.547301615415098, -84.59241411185369),
                    (30.546202075441435, -84.59271451926335),
                    (30.544668242576485, -84.5932080457221),
                    (30.54376271541383, -84.59354063963994),
                    (30.542644654919826, -84.5940019795905),
                    (30.54223808427564, -84.5941629121314)]
        
        site_polygon = [(30.54138236099163, -84.58530016492314),
                        (30.541354639959625, -84.59722164504568),
                        (30.542666760130615, -84.59876522705666),
                        (30.543221171775826, -84.60335716889016),
                        (30.546676933001226, -84.60355028793923),
                        (30.552463858724003, -84.60195898425629),
                        (30.552759515491434, -84.59459900271942),
                        (30.55118882859853, -84.59444879901459),
                        (30.5510409979353, -84.59316133868744),
                        (30.54913765803747, -84.59335445773651),
                        (30.549008303486243, -84.5847825310166),
                        (30.541377558327387, -84.58462391262817)]
        
        polygons = [
            (site_polygon, {'outline_color': 'white', 'border_width': 3, 'name': 'Overall Site'}),
            (field1, {'outline_color': 'orange red', 'border_width': 3, 'name': 'Field 1'}),
            (field2, {'outline_color': 'cyan', 'border_width': 3, 'name': 'Field 2'}),
            (field3, {'outline_color': 'MediumPurple1', 'border_width': 3, 'name': 'Field 3'})]
        
        if index == 0:
            self.set_polygons(polygons)
        else:
            self.map_widget.delete_all_polygon()
            
    def toggle_button_callback(self):
        if self.toggle_flag%2==0:
            self.toggle_polygons(0)
        else:
            self.toggle_polygons(1)
        self.toggle_flag = self.toggle_flag+1
        
    def message_callback(self):
        values=self.values
        
        messages = []
        for i in Device_Lookup:
            code = get_todays_messages(Device_Lookup[i])
            messages.append(code)
        
         # for testing
        #messages = [5,1,6]
            
        self.MessageFrame = MyMessageBarFrame(self.tabview.tab("Messages"), title = "Received Messages", 
                                             values = values, message_code = messages)
        self.MessageFrame.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        
    
    def voltage_callback(self):
        values=self.values
        voltages = []
        for i in Device_Lookup:
            voltage = get_latest_voltage(Device_Lookup[i])
            voltages.append(voltage)
        
        self.batterybars = MyProgressBarFrame(self.tabview.tab("Properties"), title = "Device Charge", values=values,
                                             progress = voltages, max_volt = 4.85, min_volt = 4.55)
        self.batterybars.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        
        times = []
        for i in Device_Lookup:
            time = extract_times(Device_Lookup[i], 'field1')
            times.append(time)
            
        self.elapsedframe = MyElapsedTimeFrame(self.tabview.tab("Properties"), title = "Time Since Receive", values=values,
                                             times = times)
        self.elapsedframe.grid(pady=(20, 0), padx=(20, 20), row=2, column=0)
        
    def status_callback(self):
        warning_image_path = os.path.join(os.getcwd(), "warning.jpg")
        warning_image = customtkinter.CTkImage(Image.open(warning_image_path),
                                  size=(17, 15))
        
        checkmark_image_path = os.path.join(os.getcwd(), "checkmark.jpg")
        checkmark_image = customtkinter.CTkImage(Image.open(checkmark_image_path),
                                  size=(20, 15))
        
        values=self.values
        
        statuses = []
        boundary_statuses = []
        for i in Device_Lookup:
            status = get_emergency_status(Device_Lookup[i])
            statuses.append(status)
            
            boundary_status = get_boundary_status(Device_Lookup[i])
            boundary_statuses.append(boundary_status)
            
        # for testing
        #statuses = [0,1,0]
            
        total_statuses = np.zeros(len(Device_Lookup))
        for i in range(len(Device_Lookup)):
            total_statuses[i] = statuses[i] + boundary_statuses[i]
        
        self.emergencyframe = MyEmergencyStatusFrame(self.tabview.tab("Statuses"), title = "Emergency Statuses", 
                                                     values = values, statuses = statuses, 
                                                     image1=warning_image, image2=checkmark_image)
        self.emergencyframe.grid(pady=(20, 0), padx=(20, 20), row=1, column=0)
        
        self.outofboundframe = MyEmergencyStatusFrame(self.tabview.tab("Statuses"), title = "Boundary Status", 
                                                     values = values, statuses = boundary_statuses, 
                                                     image1=warning_image, image2=checkmark_image)
        self.outofboundframe.grid(pady=(20, 0), padx=(20, 20), row=2, column=0)
        
        self.emergency_textbox.delete("0.0", "end")
        
        total_em = 0
        device_hone = 0
        for i, val in enumerate(total_statuses):
            if val > 0:
                total_em = total_em+1
                string = "               Device {} \n".format(i+1)
                self.emergency_textbox.insert(index = "1.0",text=string)
                device_hone = i+1
                
        if total_em == 0:
            self.emergency_textbox.insert(index = "1.0",text="               No Device \n      Requires Assistance")
            
        if device_hone != 0:
            channelID = Device_Lookup["Device {}".format(device_hone)]
            latitude = extract_vals(channelID, 'field2')
            longitude = extract_vals(channelID, 'field1')
            if latitude != 0 and longitude != 0:
                self.map_widget.set_position(latitude, longitude)
                self.map_widget.set_zoom(18)
                self.map_widget.set_marker(latitude, longitude, text = "Device {} Requires Assistance!".format(device_hone),
                                                       marker_color_circle = 'red',
                                                       marker_color_outside= 'red2',text_color = 'white')
        
    def map_path_button_callback(self):
        self.map_widget.delete_all_path()
    
        #data, vals = getGPSarray(self.checkbox_frame_1.get())
        date_string = self.path_textbox.get(index1="0.0", index2="end")
        date_string = date_string.translate({ord(c): None for c in string.whitespace})
        stringy = self.checkbox_frame_1.get()
        data, vals = getGPSarray_on_day(stringy, date_string)
        #print(data)
        data_formatted = getGPSarraysFormatted(data)
        #print(data_formatted)
        #print(vals)

        # Define path colors and names for each device
        device_paths = {
            1: {'color': 'orchid1', 'name': 'Device 1'},
            2: {'color': 'RoyalBlue1', 'name': 'Device 2'},
            3: {'color': 'lawn green', 'name': 'Device 3'},
            4: {'color': 'white', 'name': 'Device 4'},
            5: {'color': 'white', 'name': 'Device 5'},
            6: {'color': 'white', 'name': 'Device 6'},
            7: {'color': 'white', 'name': 'Device 7'},
            8: {'color': 'white', 'name': 'Device 8'},
        }

        for k, val in enumerate(vals):
            if val in device_paths: #if val in device_paths and data_formatted[k] != []:
                device_info = device_paths[val]
                for i in range(len(stringy)):
                    if stringy[i] == device_info['name']:
                        index = i
                path = self.map_widget.set_path([(2,2), (1,1)], color=device_info['color'], name=device_info['name'])
                for i in range(np.shape(data_formatted[index])[0]):
                    if float(data_formatted[index][i][1]) != 0.0 and float(data_formatted[index][i][1]) != 0.0:
                        path.add_position(float(data_formatted[index][i][1]), float(data_formatted[index][i][0]))
                path.remove_position(2,2)
                path.remove_position(1,1)
            
    def map_latest_button_callback(self):
        self.map_widget.delete_all_marker()
        
        data, vals = getGPSarray(self.checkbox_frame_1.get())
        
        display_parameters = {
            1: {'channelID': 2496188,'color': 'orchid1', 'color2': 'purple', 'name': 'Device 1'},
            2: {'channelID': 2496183,'color': 'RoyalBlue1', 'color2': 'blue', 'name': 'Device 2'},
            3: {'channelID': 2496176,'color': 'lawn green', 'color2': 'green', 'name': 'Device 3'},
            4: {'channelID': 2496176,'color': 'white', 'color2': 'white', 'name': 'Device 4'},
            5: {'channelID': 2496176,'color': 'white', 'color2': 'white', 'name': 'Device 5'},
            6: {'channelID': 2496176,'color': 'white', 'color2': 'white', 'name': 'Device 6'},
            7: {'channelID': 2496176,'color': 'white', 'color2': 'white', 'name': 'Device 7'},
            8: {'channelID': 2496176,'color': 'white', 'color2': 'white', 'name': 'Device 8'},
        }
        
        for val in vals:
            if val in display_parameters:
                device_info = display_parameters[val]
                latitude = extract_vals(device_info['channelID'], 'field2')
                longitude = extract_vals(device_info['channelID'], 'field1')
                if latitude == 0 or longitude == 0:
                    data1 = getThingSpeakDataOverTime(device_info['channelID'],'field1')
                    data2 = getThingSpeakDataOverTime(device_info['channelID'],'field2')

                    for i in range(len(data1)):
                        if data1[i] == None:
                            data1[i] = 0
                        data1[i] = float(data1[i])

                    for i in range(len(data2)):
                        if data2[i] == None:
                            data2[i] = 0
                        data2[i] = float(data2[i])

                    data1 = [x for x in data1 if x != 0.0]
                    data2 = [x for x in data2 if x != 0.0]
                    longitude = data1[-1]
                    latitude = data2[-1]
                marker = self.map_widget.set_marker(latitude, longitude, text = device_info['name'],
                                                                   marker_color_circle = device_info['color2'],
                                                                   marker_color_outside= device_info['color'],
                                                                   text_color = 'white')
            

    def change_map(self, new_map: str):
        if new_map == "OpenStreetMap":
            self.map_widget.set_tile_server("https://a.tile.openstreetmap.org/{z}/{x}/{y}.png")
        elif new_map == "Google normal":
            self.map_widget.set_tile_server("https://mt0.google.com/vt/lyrs=m&hl=en&x={x}&y={y}&z={z}&s=Ga", max_zoom=22)
        elif new_map == "Google satellite":
            self.map_widget.set_tile_server("https://mt0.google.com/vt/lyrs=s&hl=en&x={x}&y={y}&z={z}&s=Ga", max_zoom=22)

    def on_closing(self, event=0):
        self.destroy()

    def start(self):
        self.mainloop()


if __name__ == "__main__":
    app = App()
    app.start()

https://cs111.wellesley.edu/archive/cs111_fall14/public_html/labs/lab12/tkintercolor.html

